In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [4]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os

In [5]:
args={"dataset":"C:/Users/naveen.dagar/start_learning_python/covid19-multi-level/dataset",
     "model":"C:/Users/naveen.dagar/start_learning_python/covid19-multi-level/covid19.model",
     "labelbin":"C:/Users/naveen.dagar/start_learning_python/covid19-multi-level/mlb.pickle",
     "plot":"C:/Users/naveen.dagar/start_learning_python/covid19-multi-level/plot.png"}

In [6]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (224, 224, 3)

In [7]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)

# initialize the data and labels
data = []
labels = []

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)

    # extract set of class labels from the image path and update the
    # labels list
    l = label = imagePath.split(os.path.sep)[-2].split("_")
    labels.append(l)

[INFO] loading images...


In [8]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
    len(imagePaths), data.nbytes / (1024 * 1000.0)))

# binarize the labels using scikit-learn's special multi-label
# binarizer implementation
print("[INFO] class labels:")
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

# loop over each of the possible class labels and show them
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i + 1, label))

[INFO] data matrix: 298 images (350.45MB)
[INFO] class labels:
1. covid
2. normal


In [9]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.2, random_state=42)
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [10]:
# initialize the model using a sigmoid activation as the final layer
# in the network so we can perform multi-label classification
print("[INFO] compiling model...")
model = SmallerVGGNet.build(
    width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
    depth=IMAGE_DIMS[2], classes=len(mlb.classes_),
    finalAct="sigmoid")
# initialize the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

[INFO] compiling model...
Instructions for updating:
Colocations handled automatically by placer.


In [11]:
# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Epoch 1/75
7/7 [==============================] - 130s 19s/step - loss: 1.1066 - accuracy: 0.5898 - val_loss: 0.4663 - val_accuracy: 0.9333
Epoch 2/75
7/7 [==============================] - 123s 18s/step - loss: 0.7552 - accuracy: 0.6238 - val_loss: 0.2865 - val_accuracy: 0.9333
Epoch 3/75
7/7 [==============================] - 122s 17s/step - loss: 0.6497 - accuracy: 0.6917 - val_loss: 0.3010 - val_accuracy: 0.9333
Epoch 4/75
7/7 [==============================] - 135s 19s/step - loss: 0.5580 - accuracy: 0.7188 - val_loss: 0.3975 - val_accuracy: 0.9333
Epoch 5/75
7/7 [==============================] - 124s 18s/step - loss: 0.4960 - accuracy: 0.7840 - val_loss: 0.2820 - val_accuracy: 0.9333
Epoch 6/75
7/7 [==============================] - 127s 18s/step - loss: 0.4271 - accuracy: 0.8422 - val_loss: 0.3203 - val_accuracy: 0.9417
Epoch 7/75
7/7 [==============================] - 129s 18s/step - loss: 0.4076 - accu

Epoch 57/75
7/7 [==============================] - 115s 16s/step - loss: 0.0423 - accuracy: 0.9879 - val_loss: 0.2509 - val_accuracy: 0.9250
Epoch 58/75
7/7 [==============================] - 117s 17s/step - loss: 0.0745 - accuracy: 0.9757 - val_loss: 0.3952 - val_accuracy: 0.9083
Epoch 59/75
7/7 [==============================] - 128s 18s/step - loss: 0.0540 - accuracy: 0.9854 - val_loss: 0.2900 - val_accuracy: 0.9250
Epoch 60/75
7/7 [==============================] - 126s 18s/step - loss: 0.0667 - accuracy: 0.9754 - val_loss: 0.2640 - val_accuracy: 0.9500
Epoch 61/75
7/7 [==============================] - 108s 15s/step - loss: 0.0345 - accuracy: 0.9840 - val_loss: 0.2679 - val_accuracy: 0.9250
Epoch 62/75
7/7 [==============================] - 127s 18s/step - loss: 0.0159 - accuracy: 0.9978 - val_loss: 0.2460 - val_accuracy: 0.9333
Epoch 63/75
7/7 [==============================] - 117s 17s/step - loss: 0.0631 - accuracy: 0.9806 - val_loss: 0.2147 - val_accuracy: 0.9500
Epoch 64/75
7

In [12]:
# save the model to disk
print("[INFO] serializing CNN network...")
model.save(args["model"])

# save the multi-label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open(args["labelbin"], "wb")
f.write(pickle.dumps(mlb))
f.close()

[INFO] serializing CNN network...
[INFO] serializing label binarizer...


In [13]:
# plot the training loss and accuracy
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(args["plot"])